In [1]:
!pip install mediapipe

  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   ------- -------------------------------- 9.4/51.0 MB 45.2 MB/s eta 0:00:01
   --------------- ------------------------ 20.2/51.0 MB 49.1 MB/s eta 0:00:01
   ------------------------ --------------- 31.2/51.0 MB 49.5 MB/s eta 0:00:01
   -------------------------------- ------- 41.9/51.0 MB 49.4 MB/s eta 0:00:01
   ---------------------------------------  50.9/51.0 MB 50.6 MB/s eta 0:00:01
   ---------------------------------------- 51.0/51.0 MB 42.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ------------ --------------------------- 5.0/15.8 MB 23.2 MB/s eta 0:00:01
   --------------- ------------------------ 6.3/15.8 MB 25.8 MB/s eta 0:00:01
   --------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
!pip install comtypes pycaw
# !pip install ctypes


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
import cv2
import mediapipe as mp
from math import hypot

import numpy as np

cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

min_vol, max_vol = volume.GetVolumeRange()[:2]




while True:
    success,img = cap.read()
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    lmList = []
    if results.multi_hand_landmarks:
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark):
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy])
            mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)

    if lmList != []:
        x1,y1 = lmList[4][1],lmList[4][2]
        x2,y2 = lmList[8][1],lmList[8][2]

        cv2.circle(img,(x1,y1),4,(255,0,0),cv2.FILLED)
        cv2.circle(img,(x2,y2),4,(255,0,0),cv2.FILLED)
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)

        length = hypot(x2-x1,y2-y1)

        value = np.interp(length,[15,300],[min_vol,max_vol])
        volume.SetMasterVolumeLevel(value, None)



    cv2.imshow('Image',img)
    if cv2.waitKey(1) & 0xff==ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

I've added a cell to install the required libraries. I've also removed the `ctypes` import from the next cell as it's a built-in library and doesn't need to be imported explicitly.